In [1]:
import sys
import vtk

In [2]:
ug_fn = "/home/marsdenlab/projects/SV/UQ/data/mesh_ug.vtk"
pd_fn = "/home/marsdenlab/projects/SV/UQ/data/mesh_pd.vtk"

wall_ids = [1,2,3]
caps = {
    4:"aorta",
    5:"aorta_2",
    6:"right_iliac"
}

OUT_DIR  = "/home/marsdenlab/projects/SV/UQ/data/mesh-complete"
SURF_DIR = OUT_DIR+"/mesh-surfaces"

ug_out_fn = OUT_DIR+"/mesh.vtu"
pd_out_fn = OUT_DIR+"/exterior.vtp"

walls_combined_fn = OUT_DIR+"/walls_combined.vtp"

In [3]:
def read_vtk(fn):
    reader = vtk.vtkGenericDataObjectReader()
    reader.SetFileName(fn)
    reader.Update()
    return reader.GetOutput()

def thresholdPolyData(poly, attr, threshold):
    """
    Get the polydata after thresholding based on the input attribute
    Args:
        poly: vtk PolyData to apply threshold
        atrr: attribute of the cell array
        threshold: (min, max)
    Returns:
        output: vtk PolyData after thresholding
    """
    surface_thresh = vtk.vtkThreshold()
    surface_thresh.SetInputData(poly)
    surface_thresh.SetInputArrayToProcess(0,0,0,1,attr)
    surface_thresh.ThresholdBetween(*threshold)
    surface_thresh.Update()
    surf_filter = vtk.vtkDataSetSurfaceFilter()
    surf_filter.SetInputData(surface_thresh.GetOutput())
    surf_filter.Update()
    return surf_filter.GetOutput()

In [4]:
ug = read_vtk(ug_fn)

ugwriter = vtk.vtkXMLUnstructuredGridWriter()
ugwriter.SetCompressorTypeToZLib()
ugwriter.EncodeAppendedDataOff()
ugwriter.SetInputDataObject(ug)
ugwriter.SetFileName(ug_out_fn)
ugwriter.Write()

1

In [5]:
pd = read_vtk(pd_fn)

pdwriter = vtk.vtkXMLPolyDataWriter()
pdwriter.SetCompressorTypeToZLib()
pdwriter.EncodeAppendedDataOff()
pdwriter.SetInputDataObject(pd)
pdwriter.SetFileName(pd_out_fn)
pdwriter.Write()

1

In [6]:
#walls combined
appender = vtk.vtkAppendPolyData()
appender.UserManagedInputsOff()
for id in wall_ids:
    w_pd = thresholdPolyData(pd, "ModelFaceID", (id,id))
    appender.AddInputData(w_pd)
    
appender.Update()

cleaner =vtk.vtkCleanPolyData()
cleaner.PointMergingOn()
cleaner.PieceInvariantOff()
cleaner.SetInputDataObject(appender.GetOutput())
cleaner.Update()

pdwriter.SetFileName(walls_combined_fn)
pdwriter.SetInputDataObject(cleaner.GetOutput())
pdwriter.Write()

1

In [7]:
for k,v in caps.items():
    
    c_pd = thresholdPolyData(pd, "ModelFaceID", (k,k))
    fn   = SURF_DIR + '/cap_'+v+'.vtp'
    
    pdwriter.SetFileName(fn)
    pdwriter.SetInputDataObject(c_pd)
    pdwriter.Write()